In [ ]:
!pip install mlflow xgboost "scikit-learn==1.3.1"

In [ ]:
!pip install --upgrade --quiet sagemaker>=2.215.0

In [ ]:
!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0

In [ ]:
! pip install scipy==1.8.0

In [ ]:
import mlflow.xgboost
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pandas as pd
import os
import mlflow
from mlflow import MlflowClient
import sagemaker
from sagemaker.serve import SchemaBuilder
from sagemaker.serve import ModelBuilder
from sagemaker.serve.mode.function_pointers import Mode
import numpy as np

In [ ]:
# Step 1: Load data
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

In [ ]:
tracking_uri='arn:aws:sagemaker:us-east-1:750573229682:mlflow-tracking-server/mlflow-tracking-server-sagemaker-poc'
mlflow.set_tracking_uri(tracking_uri) 

In [ ]:
# Step 3: Log model in MLflow
mlflow.set_experiment("xgboost_iris_experiment")
with mlflow.start_run() as run:
    # Log model
    mlflow.xgboost.log_model(model, artifact_path="xgboost_model")
    print(f"Model logged in MLflow with run ID: {mlflow.active_run().info.run_id}")
    run_id = run.info.run_id

In [ ]:
## testing through mlflow  
## Step 4: Load model from MLflow
## run_id = mlflow.active_run().info.run_id
# model_uri = f"runs:/{run_id}/xgboost_model"
# loaded_model = mlflow.xgboost.load_model(model_uri)

## Step 5: Predict on multiple samples one by one
# samples = X_test.iloc[:5]  # Select first 5 samples from the test set

# print("Predictions:")
# for i, sample in samples.iterrows():
#     sample = sample.values.reshape(1, -1)
#     prediction = loaded_model.predict(sample)
#     print(f"Sample {i}: Predicted class = {prediction[0]}")

## Evaluate model accuracy
# y_pred = loaded_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Model accuracy on test set: {accuracy}")

In [ ]:
# create Schema
iris_input = np.array([1.0, 2.0, 3.0, 4.0]).reshape(1, -1)
iris_output = 1
iris_schema_builder = SchemaBuilder(
    sample_input=iris_input,
    sample_output=iris_output,
)

In [ ]:
role = 'arn:aws:iam::750573229682:role/service-role/AmazonSageMaker-ExecutionRole-20241211T150457'

In [ ]:
model_path='s3://sagemaker-studio-750573229682-fffkyjouino/models/67/ac9a6bbc157d485a98880e6861ff554c/artifacts/xgboost_model'

In [ ]:
# Create a ModelBuilder instance for deploying the model
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=iris_schema_builder,
    model_metadata={
        "MLFLOW_MODEL_PATH": model_path, 
    },
    role_arn=role  
)

In [ ]:
built_model = model_builder.build()

In [ ]:
# Deploy the model
predictor = built_model.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.large"
    # Type of SageMaker instance
)

In [ ]:
iris_input = np.array([1.0, 2.0, 3.0, 4.0]).reshape(1, -1)
predictor.predict(iris_input)